# Example Neuron from Scratch

This code defines a simple artificial neuron and uses it to calculate the output of the neuron given certain inputs and weights. Here's a breakdown of the code:   

1. `import math`: This imports the math module, which provides mathematical functions like `exp` (exponential function).   
2. `sigmoid(x)`: This function implements the sigmoid activation function, which is commonly used in artificial neural networks. It takes a single argument `x` and returns the sigmoid of `x`, calculated as `1 / (1 + math.exp(-x))`.   
3. `activate(inputs, weights)`: This function calculates the output of the neuron given inputs and corresponding weights. It first computes the net input `h` by summing the element-wise multiplication of inputs and weights. Then, it returns the output of the neuron by applying the sigmoid activation function to `h`.   
4. `inputs` and `weights`: These are the input values and corresponding weights for the neuron.   
5. `output = activate(inputs, weights)`: This line calculates the output of the neuron using the `activate` function with the provided inputs and weights.   
6. `print(output)`: This prints the output of the neuron to the console.   
   
In this example, the output of the neuron is calculated as the sigmoid of the sum of `(0.5 \* 0.4) + (0.3 \* 0.7) + (0.2 \* 0.2)`, which is approximately `0.610639233949222`.   

In [3]:
import math


def sigmoid(x):
    y=1/(1+math.exp(-x))
    return y

def activate(inputs, weights):
    #perform net input
    h=0
    for x,w in zip(inputs, weights):
        h+=x*w
    
    return sigmoid(h)

inputs = [0.5, 0.3, 0.2]
weights = [0.4, 0.7, 0.2]

output = activate(inputs, weights)
print(output)

0.610639233949222


## Multilayer Perceptron

Overall, this code provides a basic implementation of a feedforward neural network (specifically, a multilayer perceptron), demonstrating the concepts of weight initialization, forward propagation, and activation functions.

In [9]:
import numpy as np


class MLP(object):

    """A Multilayer Perceptron class.
    """

    def __init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2):
        """Constructor for the MLP. Takes the number of inputs,
            a variable number of hidden layers, and number of outputs

        Args:
            num_inputs (int): Number of inputs
            hidden_layers (list): A list of ints for the hidden layers
            num_outputs (int): Number of outputs
        """

        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs

        # create a generic representation of the layers
        layers = [num_inputs] + hidden_layers + [num_outputs]

        # create random connection weights for the layers
        weights = []
        for i in range(len(layers)-1):
            w = np.random.rand(layers[i], layers[i+1])
            weights.append(w)
        self.weights = weights


    def forward_propagate(self, inputs):
        """Computes forward propagation of the network based on input signals.

        Args:
            inputs (ndarray): Input signals
        Returns:
            activations (ndarray): Output values
        """

        # the input layer activation is just the input itself
        activations = inputs

        # iterate through the network layers
        for w in self.weights:

            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)

            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)

        # return output layer activation
        return activations


    def _sigmoid(self, x):
        """Sigmoid activation function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """
        
        y = 1.0 / (1 + np.exp(-x))
        return y


if __name__ == "__main__":

    # create a Multilayer Perceptron
    mlp = MLP()

    # set random values for network's input
    inputs = np.random.rand(mlp.num_inputs)

    # perform forward propagation
    output = mlp.forward_propagate(inputs)
    
    print(f"inputs: {inputs}")
    print("Network activation: {}".format(output))

inputs: [0.45574792 0.09969827 0.0701312 ]
Network activation: [0.57579766 0.69735376]


## (MLP) Class

This Python class represents a Multilayer Perceptron (MLP), a type of artificial neural network. The MLP is initialized with a specified number of inputs, a variable number of hidden layers, and a specified number of outputs. The class includes methods for initializing the network with random weights and performing forward propagation to compute output values based on input signals.

### `__init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2)`

- Constructor method for the MLP.
- Initializes the MLP with the specified number of inputs, hidden layers, and outputs.
- Creates a generic representation of the network layers and initializes random connection weights for each layer.

### `forward_propagate(self, inputs)`

- Method to compute forward propagation of the network based on input signals.
- Takes input signals as input and returns output values.
- Iterates through the network layers, calculates matrix multiplication between the previous activation and weight matrix, and applies the sigmoid activation function to the result.

### `_sigmoid(self, x)`

- Private method for the sigmoid activation function.
- Takes a single value `x` and returns the sigmoid of `x`.

### `if __name__ == "__main__":`

- Main block of code to create and test the MLP.
- Creates an instance of the MLP class.
- Sets random values for the network's input.
- Performs forward propagation and prints the inputs and network activation.

The `MLP` class provides a basic implementation of a multilayer perceptron neural network for educational purposes. Additional functionality, such as training the network using backpropagation, can be added for more complex applications.


# Multilayer Perceptron

Overall, the MLP class provides a basic implementation of a feedforward neural network with backpropagation. It can be used to understand the concepts of neural network architecture, forward and backward propagation, and gradient descent optimization.

In [11]:
import numpy as np
from random import random


class MLP(object):
    """A Multilayer Perceptron class.
    """

    def __init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2):
        """Constructor for the MLP. Takes the number of inputs,
            a variable number of hidden layers, and number of outputs

        Args:
            num_inputs (int): Number of inputs
            hidden_layers (list): A list of ints for the hidden layers
            num_outputs (int): Number of outputs
        """

        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs

        # create a generic representation of the layers
        layers = [num_inputs] + hidden_layers + [num_outputs]

        # create random connection weights for the layers
        weights = []
        for i in range(len(layers) - 1):
            w = np.random.rand(layers[i], layers[i + 1])
            weights.append(w)
        self.weights = weights

        # save derivatives per layer
        derivatives = []
        for i in range(len(layers) - 1):
            d = np.zeros((layers[i], layers[i + 1]))
            derivatives.append(d)
        self.derivatives = derivatives

        # save activations per layer
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations


    def forward_propagate(self, inputs):
        """Computes forward propagation of the network based on input signals.

        Args:
            inputs (ndarray): Input signals
        Returns:
            activations (ndarray): Output values
        """

        # the input layer activation is just the input itself
        activations = inputs

        # save the activations for backpropogation
        self.activations[0] = activations

        # iterate through the network layers
        for i, w in enumerate(self.weights):
            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)

            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)

            # save the activations for backpropogation
            self.activations[i + 1] = activations

        # return output layer activation
        return activations


    def back_propagate(self, error):
        """Backpropogates an error signal.
        Args:
            error (ndarray): The error to backprop.
        Returns:
            error (ndarray): The final error of the input
        """

        # iterate backwards through the network layers
        for i in reversed(range(len(self.derivatives))):

            # get activation for previous layer
            activations = self.activations[i+1]

            # apply sigmoid derivative function
            delta = error * self._sigmoid_derivative(activations)

            # reshape delta as to have it as a 2d array
            delta_re = delta.reshape(delta.shape[0], -1).T

            # get activations for current layer
            current_activations = self.activations[i]

            # reshape activations as to have them as a 2d column matrix
            current_activations = current_activations.reshape(current_activations.shape[0],-1)

            # save derivative after applying matrix multiplication
            self.derivatives[i] = np.dot(current_activations, delta_re)

            # backpropogate the next error
            error = np.dot(delta, self.weights[i].T)


    def train(self, inputs, targets, epochs, learning_rate):
        """Trains model running forward prop and backprop
        Args:
            inputs (ndarray): X
            targets (ndarray): Y
            epochs (int): Num. epochs we want to train the network for
            learning_rate (float): Step to apply to gradient descent
        """
        # now enter the training loop
        for i in range(epochs):
            sum_errors = 0

            # iterate through all the training data
            for j, input in enumerate(inputs):
                target = targets[j]

                # activate the network!
                output = self.forward_propagate(input)

                error = target - output

                self.back_propagate(error)

                # now perform gradient descent on the derivatives
                # (this will update the weights
                self.gradient_descent(learning_rate)

                # keep track of the MSE for reporting later
                sum_errors += self._mse(target, output)

            # Epoch complete, report the training error
            print("Error: {} at epoch {}".format(sum_errors / len(items), i+1))

        print("Training complete!")
        print("=====")


    def gradient_descent(self, learningRate=1):
        """Learns by descending the gradient
        Args:
            learningRate (float): How fast to learn.
        """
        # update the weights by stepping down the gradient
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights += derivatives * learningRate


    def _sigmoid(self, x):
        """Sigmoid activation function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """

        y = 1.0 / (1 + np.exp(-x))
        return y


    def _sigmoid_derivative(self, x):
        """Sigmoid derivative function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """
        return x * (1.0 - x)


    def _mse(self, target, output):
        """Mean Squared Error loss function
        Args:
            target (ndarray): The ground trut
            output (ndarray): The predicted values
        Returns:
            (float): Output
        """
        return np.average((target - output) ** 2)


if __name__ == "__main__":

    # create a dataset to train a network for the sum operation
    items = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
    targets = np.array([[i[0] + i[1]] for i in items])

    # create a Multilayer Perceptron with one hidden layer
    mlp = MLP(2, [5], 1)

    # train network
    mlp.train(items, targets, 50, 0.1)

    # create dummy data
    input = np.array([0.3, 0.1])
    target = np.array([0.4])

    # get a prediction
    output = mlp.forward_propagate(input)

    print()
    print("Our network believes that {} + {} is equal to {}".format(input[0], input[1], output[0]))

Error: 0.049255020742547624 at epoch 1
Error: 0.04042887313304946 at epoch 2
Error: 0.040118170336748145 at epoch 3
Error: 0.039763507880993094 at epoch 4
Error: 0.039338366672448825 at epoch 5
Error: 0.03881259290383523 at epoch 6
Error: 0.03815125808391212 at epoch 7
Error: 0.037314281492065504 at epoch 8
Error: 0.03625741159840348 at epoch 9
Error: 0.034935529650723114 at epoch 10
Error: 0.033309336519457 at epoch 11
Error: 0.03135585565344104 at epoch 12
Error: 0.029081254754785173 at epoch 13
Error: 0.026531446894426994 at epoch 14
Error: 0.023793844094370242 at epoch 15
Error: 0.020985779995223375 at epoch 16
Error: 0.018232081718472043 at epoch 17
Error: 0.015641048656065214 at epoch 18
Error: 0.013288516760431905 at epoch 19
Error: 0.011213821639420224 at epoch 20
Error: 0.009425073034414918 at epoch 21
Error: 0.007908570529088743 at epoch 22
Error: 0.006638159804844565 at epoch 23
Error: 0.005582474023290745 at epoch 24
Error: 0.00470967445820428 at epoch 25
Error: 0.003990117

## Multilayer Perceptron (MLP) Class for Neural Network Implementation

This Python class implements a simple Multilayer Perceptron (MLP) for a neural network. It includes methods for initializing the network, performing forward propagation, backpropagation, and training the network using gradient descent.

### `__init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2)`

- Constructor method for the MLP class.
- Initializes the MLP with the specified number of inputs, hidden layers, and outputs.
- Creates random connection weights for each layer and initializes storage for derivatives and activations per layer.

### `forward_propagate(self, inputs)`

- Method to compute forward propagation of the network based on input signals.
- Takes input signals as input and returns output values.
- Iterates through the network layers, calculates the net inputs, applies the sigmoid activation function, and saves the activations for backpropagation.

### `back_propagate(self, error)`

- Method to backpropagate an error signal through the network.
- Updates the derivatives for each layer based on the error and activations.
- Backpropagates the error to the previous layer for further adjustment of weights.

### `train(self, inputs, targets, epochs, learning_rate)`

- Method to train the network using forward propagation, backpropagation, and gradient descent.
- Takes training data, number of epochs, and learning rate as inputs.
- Iterates through the training data, calculates the error, backpropagates the error, and updates the weights using gradient descent.

### `gradient_descent(self, learningRate=1)`

- Method to update the weights of the network using gradient descent.
- Takes the learning rate as input and updates the weights based on the derivatives.

### `_sigmoid(self, x)`

- Sigmoid activation function used in the network.
- Takes a value `x` and returns the sigmoid of `x`.

### `_sigmoid_derivative(self, x)`

- Sigmoid derivative function used in backpropagation.
- Takes a value `x` and returns the derivative of the sigmoid function.

### `_mse(self, target, output)`

- Mean Squared Error (MSE) loss function.
- Takes the target values and predicted output values as inputs and returns the average squared error.

### `if __name__ == "__main__":`

- Main block of code to create and test the MLP.
- Creates a dataset for training the network to perform the sum operation.
- Creates an instance of the MLP class with one hidden layer.
- Trains the network using the dataset.
- Tests the network with dummy data and prints the predicted output.

The `MLP` class provides a basic implementation of a feedforward neural network with one hidden layer for educational purposes. Additional functionality, such as different activation functions, layer types, and optimization algorithms, can be added for more complex applications.
